In [ ]:
import os
import glob
import requests
import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import h5py
from zeep import Client, Settings
import zeep
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.dates import DateFormatter
import json as js

In [ ]:
areaName = 'WalnutGulch'
#areaName = 'LittleRiver'

match areaName:
    case 'WalnutGulch':
        path     = 62
        frame    = 620
    case 'LittleRiver':
        path     = 47
        frame    = 620

In [ ]:
#Set some locations and variables
workDir                      = '/scratch/rlohman/'+areaName+'/Path'+str(path)+'Frame'+str(frame)+'/'
fgcovDir                     = workDir+'filtgcov/'
NRCS_AWDB_SOAP_WSDL_URL      = "https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL"
NRCS_AWDB_REST_DATA_ENDPOINT = "https://wcc.sc.egov.usda.gov/awdbRestApi/services/v1/data"
ELEMENT_CODES                = ("SMS:*")# Soil Moisture PERCENT
DURATION                     = "DAILY"
retNames                     = ['DSG','PMI','TSR']
nret                         = len(retNames)
maxdays                      = 7 #use closest in situ measurement to retrieval date, with max time difference = maxdays 
targerr                      = 6 #UBRMSE target range in percent

In [ ]:
#Get list of gcovs, ordered by date (will need to replace with ASF search by track/frame, when available)
gcovs     = glob.glob(fgcovDir+'2*h5')
dates     = np.array([datetime.datetime.strptime(x[-11:-3],'%Y%m%d') for x in gcovs])
sortindex = np.argsort(dates)

dates     = [dates[x] for x in sortindex]
gcovs     = [gcovs[x] for x in sortindex]

#Get Lat/Lon for searching through sparse network
fo        = h5py.File(gcovs[0],'r')
lat       = fo['latitude'][()]
lon       = fo['longitude'][()]
s0        = fo['Sigma0_hh_aggregated'][()] #just for plotting
fo.close()
s0dB      = 10*np.log10(s0)
s01       = np.nanpercentile(s0dB,2,axis=None) #for color range below
s02       = np.nanpercentile(s0dB,98,axis=None)

In [ ]:
#API lets us search min/max long/lat
minLongitude = np.min(lon)
maxLongitude = np.max(lon)
minLatitude  = np.min(lat)
maxLatitude  = np.max(lat)
#for plotting
footprint    = np.array([[minLongitude, minLatitude],[minLongitude, maxLatitude],[maxLongitude, maxLatitude],[maxLongitude, minLatitude],[minLongitude, minLatitude]])


In [ ]:
settings         = Settings(strict=False, xml_huge_tree=True)
client           = Client(NRCS_AWDB_SOAP_WSDL_URL,settings=settings)
station_triplets = client.service.getStations(elementCds='SMS',minLatitude=minLatitude,maxLatitude=maxLatitude,minLongitude=minLongitude,maxLongitude=maxLongitude, logicalAnd=True)

data             = client.service.getStationMetadataMultiple(stationTriplets=station_triplets)
df               = pd.DataFrame.from_records(zeep.helpers.serialize_object(data))
ptlon            = df.longitude.to_numpy().astype('float')
ptlat            = df.latitude.to_numpy().astype('float')

print(str(len(station_triplets))+' stations in footprint')

In [ ]:
# Get GCOV pixel for each in situ station within frame
idx  = np.array([np.linalg.norm(x) for x in lon-ptlon]).argmin()
idy  = np.array([np.linalg.norm(x) for x in lat-ptlat]).argmin()

# Pull retrieval values from all gcovs
retr = np.zeros([len(gcovs),nret])
for i in range(len(gcovs)):
    fo=h5py.File(gcovs[i],'r')
    for j in range(nret):
        retr[i,j] =fo['Algorithm/'+retNames[j]+'/Soil_moisture'][idy,idx]
    fo.close()


In [ ]:
fig,axes = plt.subplots(1,2)

# CONUS basemap.
m = Basemap(projection='merc',llcrnrlat=24,urcrnrlat=49,llcrnrlon=-125,urcrnrlon=-66,lat_ts=35,resolution=None,ax=axes[0])
m.shadedrelief()
xpt,ypt=m(footprint[:,0],footprint[:,1])
m.plot(xpt,ypt,'g')
xpt,ypt=m(ptlon,ptlat)
m.plot(xpt,ypt,'m^')

#zoom in version
m = Basemap(projection='merc',llcrnrlat=minLatitude-0.1,urcrnrlat=maxLatitude+0.1,llcrnrlon=minLongitude-0.1,urcrnrlon=maxLongitude+0.1,lat_ts=np.mean(footprint[:,1]),resolution='i',ax=axes[1])
m.drawcoastlines()
m.drawstates()
xpt,ypt=m(footprint[:,0],footprint[:,1])
m.plot(xpt,ypt,'g')
m.pcolor(np.linspace(xpt[1],xpt[2],num=len(lon)+1),np.linspace(ypt[0],ypt[1],num=len(lat)+1),s0dB,vmin=s01,vmax=s02,cmap='gray',shading='flat')
xpt,ypt=m(ptlon,ptlat)
m.plot(xpt,ypt,'m^')
plt.show()

In [ ]:
def build_awdb_data_query_string(
    station_triplet: str,
    begin_date: str,
    end_date: str,
    elements: tuple[str, ...],
    duration: str,
):
    """Build querystring for the AWDB REST /data endpoint to get station data."""
    return "&".join(
        [
            f"stationTriplets={station_triplet}",
            f"beginDate={begin_date}",
            f"endDate={end_date}",
            "elements=SMS%3A%2A",
            f"duration={duration}",
        ]
    )

def _series_from_date_value_dicts(arr: list[dict]):
    """Utility to build a pandas Series from the {"date": ..., "value": ...} JSON records returned
    by the AWDB REST Service data/ endpoint.
    """
    dates, values = zip(*[(entry["date"], entry["value"]) for entry in arr])
    return pd.Series(values, index=dates, dtype="float")

def get_data_for_station(station_triplet: str, begin_date: datetime.date, end_date: datetime.date) -> pd.DataFrame:
    """Returns data from NRCS AWDB for a station over given date range."""
    if type(station_triplet) is list:
        if len(station_triplet) > 1 :
            station_triplet=','.join(station_triplet)

    url = (
        NRCS_AWDB_REST_DATA_ENDPOINT
        + "?"
        + build_awdb_data_query_string(
            station_triplet=station_triplet,
            begin_date=begin_date.strftime("%Y-%m-%d"),
            end_date=end_date.strftime("%Y-%m-%d"),
            elements=ELEMENT_CODES,
            duration=DURATION,
        )
    )
    print(url)
    session = requests.Session()
    response = session.get(url)

    if response.status_code == 200:
        if len(response.json())>0:
            data = {
                entry["stationElement"]["elementCode"]
                + "_"
                + entry["stationElement"]["durationName"]: _series_from_date_value_dicts(entry["values"])
                for entry in response.json()[0]["data"]
            }
            df = pd.DataFrame(data)
            df.index.name = "date"
        else:
            print('No data found')
            df = pd.DataFrame()
    else:
        print('Error at site')
        df = pd.DataFrame()
    return df

In [ ]:
begin_date         = dates[0]+timedelta(days=-7) #pull data for a slightly larger range, in case there isn't data on the exact gcov date
end_date           = dates[-1]+timedelta(days=7)
end_date           = datetime.datetime(2023,12,1)
data_df            = get_data_for_station(station_triplets[0], begin_date, end_date)

dfdates            = data_df.index.values
dfdn               = np.array([datetime.datetime.strptime(x,'%Y-%m-%d') for x in dfdates])          #datetime values for in situd data
dd                 = np.array([np.abs(x-dfdn) for x in dates]).astype('timedelta64[D]').astype(int) #delta time from gcov dates (matrix)
ddmin              = np.min(dd,1)                                                                   #closest time for each gcov date
indices            = np.argmin(dd, axis=1)                                                          #index of closest time
bigGap             = np.argwhere(ddmin>=maxdays)                                                    #which gcov dates have no in situ date within maxdays
insituDates        = dfdn[indices]                                                                  #dates of closest in situ data for each gcov
insituData         = data_df['SMS_DAILY'][indices].values                                           #in situ data for each gcov date
insituData[bigGap] = np.nan                                                                         #set values that are too many days away to NaN

minval             = np.nanmin([np.nanmin(data_df['SMS_DAILY'].values),np.nanmin(retr*100,axis=None)])-2 #for plotting
maxval             = np.nanmax([np.nanmax(data_df['SMS_DAILY'].values),np.nanmax(retr*100,axis=None)])+2

res                = retr-np.transpose(np.tile(insituData/100,(3,1)))
UBRMSE             = np.sqrt(np.nansum((res-np.nanmean(res,axis=0))**2,axis=0)/len(insituData))


print(retNames, ' UBRMSE:', UBRMSE)
print('Std Dev of res:',np.nanstd(res,ddof=1,axis=0)) #ddof=1 enforces /(n-1) instead of /n

In [ ]:
myFmt    = DateFormatter("%Y%m")
labels    =retNames.copy()
labels.insert(0,'insitu')
fig, ax1 = plt.subplots(1,2,figsize=(10,5))

color = 'tab:red'
ax1[0].set_xlabel('date')
ax1[0].set_ylabel('mv, %', color=color)
ln1=ax1[0].plot(dfdn,data_df['SMS_DAILY'], '.',color=color)
ax1[0].tick_params(axis='y', labelcolor=color)
ax1[0].set_xticks([datetime.datetime(2022,1,1),datetime.datetime(2022,7,1),datetime.datetime(2023,1,1),datetime.datetime(2023,7,1)])
ax1[0].xaxis.set_major_formatter(myFmt)
ax1[0].set_ylim([minval,maxval])

ax2 = ax1[0].twinx()             # instantiate a second Axes that shares the same x-axis
ax2.set_ylabel('retrievals')  # we already handled the x-label with ax1
ln2=ax2.plot(dates,retr*100,'.' )
ax2.tick_params(axis='y')
ax2.set_ylim([minval,maxval])
ax2.legend(ln1+ln2,labels)

ax1[1].set_xlabel('date')
ax1[1].set_ylabel('mv, %', color=color)
ln1=ax1[1].plot(dfdn,data_df['SMS_DAILY'], '.',color=color)
ax1[1].tick_params(axis='y', labelcolor=color)
ax1[1].set_xticks([datetime.datetime(2022,1,1),datetime.datetime(2022,7,1),datetime.datetime(2023,1,1),datetime.datetime(2023,7,1)])
ax1[1].xaxis.set_major_formatter(myFmt)
ax1[1].set_ylim([minval,maxval])

ax3 = ax1[1].twinx()             # instantiate a second Axes that shares the same x-axis
ax3.set_ylabel('retrievals')  # we already handled the x-label with ax1
ln2=ax3.plot(dates,(retr-np.nanmean(res,axis=0))*100,'.' )
ax3.tick_params(axis='y')
ax3.set_ylim([minval,maxval])
ax3.legend(ln1+ln2,labels)


fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
color  = 'tab:red'
labels =retNames.copy()
labels.insert(0,'+/- '+str(targerr)+'%')
labels.insert(0,'insitu')
fig, ax1 = plt.subplots(1,2,figsize=(10,5))

ax1[0].set_xlabel('date')
ax1[0].set_ylabel('mv, %', color=color)
ln1=ax1[0].plot(insituDates,insituData, '.-', color=color)
ax1[0].tick_params(axis='y', labelcolor=color)
ax1[0].set_xticks([datetime.datetime(2022,1,1),datetime.datetime(2022,7,1),datetime.datetime(2023,1,1),datetime.datetime(2023,7,1)])
ax1[0].xaxis.set_major_formatter(myFmt)
ax1[0].set_ylim([minval,maxval])

ax2 = ax1[0].twinx()  # instantiate a second Axes that shares the same x-axis
ax2.set_ylabel('retrievals, %')  # we already handled the x-label with ax1
ln2=ax2.plot(dates,retr*100 )
ax2.tick_params(axis='y')
ax2.set_ylim([minval,maxval])
ax2.legend(ln1+ln2,labels)


ax1[1].set_xlabel('date')
ax1[1].set_ylabel('mv, %', color=color)
ln1=ax1[1].plot(insituDates,insituData, '.-', color=color)
ln2=ax1[1].plot(insituDates,insituData-targerr, '--', color='lightgray')
ax1[1].plot(insituDates,insituData+targerr, '--', color='lightgray')
ax1[1].tick_params(axis='y', labelcolor=color)
ax1[1].set_xticks([datetime.datetime(2022,1,1),datetime.datetime(2022,7,1),datetime.datetime(2023,1,1),datetime.datetime(2023,7,1)])
ax1[1].xaxis.set_major_formatter(myFmt)
ax1[1].set_ylim([minval,maxval])

ax3 = ax1[1].twinx()  # instantiate a second Axes that shares the same x-axis
ax3.set_ylabel('retrievals, %')  # we already handled the x-label with ax1
ln3=ax3.plot(dates,(retr-np.nanmean(res,axis=0))*100 )
ax3.tick_params(axis='y')
ax3.set_ylim([minval,maxval])
ax3.legend(ln1+ln2+ln3,labels)


fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()